In [1]:
# Imports and definitions
import numpy as np
from collections import defaultdict
from collections import namedtuple
from dataclasses import dataclass
import urllib.request
import hashlib
import functools
import time
import random
from nacl.signing import SigningKey

In [2]:
# block class
# each block has transactions
# output has a date attached --> how to check if output has been spent? add "spent" variable?

HashPointer = namedtuple('HashPointer', ['hash', 'pointer'])
Transaction = namedtuple('Transaction', ['inputs', 'outputs', 'age'])

Input = namedtuple('Input', ['previous_tx', 'index', 'public_key'])
SignedInput = namedtuple('SignedInput', ['input', 'signed_input'])

Output = namedtuple('Output', ['public_key_hash', 'value'])

class Block:
    def __init__(self, transactions, prev, nonce, pubkey_hash):
        self.transactions = transactions
        self.prev = prev
        self.nonce = nonce
        self.pubkey_hash = pubkey_hash
        self.age = time.time()
    
    def __repr__(self):
        return f'\nBlock(\n transaction: {self.transactions},\n nonce: {self.nonce},\n prev: {self.prev})'

def bytes_value(v):
    return bytes(str(v), encoding='utf-8')

def hash_value(v):
    return hashlib.sha256(bytes_value(v)).hexdigest()

def add_block(transactions, blockchain, nonce, pubkey_hash):
    prev_hash = hash_value(blockchain)
    prev = HashPointer(prev_hash, blockchain)
    new_block = Block(transactions, prev, nonce, pubkey_hash)
    
    return new_block, hash_value(new_block)

In [3]:
# mining function
# difficulty is calculated by the product of the output of the transaction and the # of blocks where the 
# transaction output has not been spent
DIFFICULTY = int(2**(32 * 8)/10000)
# the system we described wouldn't have to mine, so i'm not sure what to do with this
def mine_for_block(transactions, pubkey_hash, blockchain):
    nonce = 0
    while True:
        nonce = nonce + 1
        # 1. construct a block that contains the transaction
        new_blockchain, final_hash = add_block(transactions, blockchain, nonce, pubkey_hash)
        # 2. find a nonce such that the new block hashes to a number less than or equal to the difficulty
        if int(final_hash, 16) <= DIFFICULTY:
            print("Found a block! Final nonce:", nonce)
            return new_blockchain, final_hash

In [4]:
# peer class
# every peer has list of peers, do their receive_block to consider candidates
# take block with greater value

class Peer():
    transactions: dict[str, Transaction]
    potential_spendable_transactions: dict[str, Transaction]
    current_block: Block
    pubkey_hash: SigningKey
        
    def __init__(self):
        self.current_block = None
        self._secret_key = SigningKey.generate()
        self.public_key = self._secret_key.verify_key
        self.pubkey_hash = hash_value(self.public_key)
        self.calc_txs()

    def calc_txs(self):
        self.transactions = {}
        self.potential_spendable_transactions = {}
        curr_block = self.current_block
        while curr_block is not None:
            for tx in curr_block.transactions:
                self.transactions[hash_value(tx)] = tx
                for output in tx.outputs:
                    if output.public_key_hash == self.pubkey_hash:
                        self.potential_spendable_transactions[hash_value(tx)] = tx
            curr_block = curr_block.prev.pointer
    
    def receive_block_candidate(self, proposed_block, block_hash):
        # validate block
        
        # calculate value of blocks
        # go thru transactions in the block and find oldest one. divide value / age (smaller => better)
        block_age = 0
        for tx in self.current_block.transactions:
            if block_age < tx.age:
                block_age = tx.age
                block_amount = sum([output.value for output in tx.outputs])
        block_val = (block_age // block_amount)
            
        prop_block_age = 0
        for tx in proposed_block.transactions:
            if prop_block_age < tx.age:
                prop_block_age = tx.age
                prop_block_amount = sum([output.value for output in tx.outputs])
        prop_block_val = (prop_block_age // prop_block_amount)
        
        
        # take block with smaller value
        if (block_val < prop_block_val):
            # add current_block to chain    
            self.current_block = proposed_block #should probably validate the block at some point, could use block validation from HW
            # update transaction registry
            self.calc_txs()
            print("Accepted new block")
        else:
            print("Rejected new block")
        # else do nothing because the current block is already stored.
        # if vals are equal it's just going to go with the proposed one. this is probably fine
            # could fix this by incorperating the nonce into the value calc, maybe lower nonce wins?
        self.calc_txs()
            
        
    def produce_block_candidate(self, peers):
        # pick the first available transaction output to spend
        tx_to_spend = None
        output_idx = None
        spendable = False
        for key, tx in self.potential_spendable_transactions.items():
            for i, output in enumerate(tx.outputs):
                if output.public_key_hash == self.pubkey_hash:
                    # output is meant for self, doesn't mean it's spendable yet
                    # set these flags because we assume that it IS until proven otherwise
                    spendable = True 
                    tx_to_spend = tx
                    output_idx = i
                    for key2, tx2 in self.transactions.items():
                        for t_input in tx2.inputs:
                            if t_input.input.previous_tx == key and t_input.input.index == output_idx:
                                # output was spent in another tx, so abandon
                                spendable = False
        
        # if such a tx exists, spend it
        if spendable:
            # Randomly break up the input into outputs 
            input_val = tx_to_spend.outputs[output_idx].value
            output_vals = []
            while input_val > 0:
                output_val = random.randint(1, input_val)
                output_vals.append(output_val)
                input_val -= output_val

            # Randomly choose who gets those outputs
            outputs = []
            for output_val in output_vals:
                outputs.append(Output(public_key_hash=peers[random.randint(0, len(peers)-1)].pubkey_hash, value=output_val))
            
            # create tx
            tx_input = Input(previous_tx=hash_value(tx_to_spend), index=output_idx, public_key=self.pubkey_hash)
            signed_input = SignedInput(input=tx_input, signed_input=self._secret_key.sign(bytes_value(tx_input)))

            new_tx = Transaction(inputs=[signed_input], outputs=outputs, age=time.time())

            # create block
            block, block_hash = mine_for_block([new_tx], self.pubkey_hash, self.current_block)
        # else use existing block which will surely lose
        else:
            block = self.current_block
            block_hash = hash_value(self.current_block)

        # return candidate
        return block, block_hash
        


In [5]:


# simulator function 
# has a dictionary of peers --> look-up table so peers can interact
# record of transactions 
def run_simulator():
    NUM_PEERS = 10

    peers: list[Peer] = [Peer() for _ in range(NUM_PEERS)]

    sim_key = SigningKey.generate()
    sim_pub_key = sim_key.verify_key
    sim_pub_key_hash = hash_value(sim_pub_key)
    # mint some coins to seed the interactions
    mint_input = Input(previous_tx="COINBASE", index=0, public_key=sim_pub_key_hash)
    outputs = [Output(public_key_hash=peer.pubkey_hash, value=10) for peer in peers]
    initial_tx = Transaction(inputs=[SignedInput(input=mint_input, signed_input=sim_key.sign(bytes_value(mint_input)))], outputs=outputs, age=time.time())

    initial_block, initial_block_hash = mine_for_block([initial_tx], sim_pub_key_hash, None)

    for peer in peers:
        peer.current_block = initial_block
        peer.calc_txs()
    
    print("BEGIN SIMULATOR...")
    for round_num in range(1, 10):
        print(f"---START ROUND {round_num}---")
        candidates = []
        hashes = []
        for peer in peers:
            candidate, candidate_hash = peer.produce_block_candidate(peers)
            candidates.append(candidate)
            hashes.append(candidate_hash)
        
        for i, (candidate, c_hash) in enumerate(zip(candidates, hashes)):
            print(f"Peer {i} proposes {c_hash}:")
            for peer in peers:
                peer.receive_block_candidate(candidate, c_hash)

        print(f"----END ROUND {round_num}----")
    print("END SIMULATOR...")
    
    # wallet addresses / keys : list of transactions that go to said address
    # dictionary of peer : address

In [6]:
run_simulator()

Found a block! Final nonce: 19615
BEGIN SIMULATOR...
---START ROUND 1---
Found a block! Final nonce: 16498
Found a block! Final nonce: 7056
Found a block! Final nonce: 4670
Found a block! Final nonce: 4833
Found a block! Final nonce: 30167
Found a block! Final nonce: 903
Found a block! Final nonce: 17933
Found a block! Final nonce: 10163
Found a block! Final nonce: 3369
Found a block! Final nonce: 21488
Peer 0 proposes 00034b17ab751bfc19a1e0e3c0ebd0b988f6ef98ee9e1cd8b23d524bcd5ffb01:
Accepted new block
Accepted new block
Accepted new block
Accepted new block
Accepted new block
Accepted new block
Accepted new block
Accepted new block
Accepted new block
Accepted new block
Peer 1 proposes 0005689cff32c8d82496bebe43de4e2975b5447365e6e295557633b3e031dd12:
Rejected new block
Rejected new block
Rejected new block
Rejected new block
Rejected new block
Rejected new block
Rejected new block
Rejected new block
Rejected new block
Rejected new block
Peer 2 proposes 000367b449a3405177f322bb16117bfd5